# Piezoelectric Active Power Harvesting for General Sensor Use
Alex Hagen

NUCL 563 - Spring 2017

April 25, 2017

## The Motivation

- Sensors have become ever-present in today's society
- Modern sensors, such as accelerometers are
    - Power hungry
    - Desired to be wireless
    - Desired to be small
    - Desired to be fast
- Wireless, non-invasive sensors are the holy grail


## The Inspiration

### A low-cost application

- Botteron published a $12.6\mathrm{\mu W}$ piezoelectric/solar sensor
- Used "power harvesting" - no active component
- $3000\mathrm{mm^{2}}$ size is cumbersome for some applications

### A medical application

- Seo published neural "dust"
- Medical, actively powered through a PZT power harvester
- Small, $1\mathrm{mm^{2}}$ size
- Powered through application of ultrasonic waveform to the body

## The Goal - A Middle Ground

- Actively powered
- Small ($\ll 3000 \mathrm{mm^{2}}$)
- Higher power ($> 12.6\mathrm{\mu W}$, goal is $466.2\mathrm{\mu W}$)
- Higher Voltage ($3.3\mathrm{V}$ or $5\mathrm{V}$ for faster microprocessor speed)

## The Design

### PCB Choice

- Following Botteron, and Illich, using PZT-5A because of its common availability
- This is often called Navy PZT II
- Disk shape chosen

## The Design
### PCB Size Calculation

- Frequency of resonance chosen at $500\mathrm{kHz}$ for availability of amplifiers

<img src="pzt_diagram.png" width=200px />

- Planar Mode ($d>5t$)
$$f_{res}=\frac{N_r}{d} \therefore d=\frac{N_r}{f_{res}}$$
$$d=2.9\mathrm{mm}$$
$$t<0.58\mathrm{mm}$$

## The Design

### Resonant Circuit Choice

<img src=rlc.png width=600px />

$$f_{res} = \frac{1}{2\pi \sqrt{LC}}$$

## The Design

### Resonant Circuit Choice

- The pzt has an internal capacitance of

$$C=\frac{\pi}{4}k_{33}\varepsilon_{0}\frac{d^{2}}{t}=0.1\mathrm{nF}$$

- Ballast the pzt with a larger capacitor to decrease the size required in the inductor (space saving)
- Ballast capacitor of $10\mathrm{\mu F}$, gives inductor size of $10\mathrm{nH}$

## The Design

### Regulator Choice

- Tried two ultra-low power boost converters - (TI's BQ25504 as per Botteron, LT's LTC3108), low success, details to follow
- Chose a voltage regulator (need to step-**down** the voltage from PZT)
- High speed required to match $500\mathrm{kHz}$
- Output of $3.6\mathrm{V}$ chosen to match a single cell LiPo battery
- Load on back end of $100\mathrm{k\Omega}$ matches common microprocessors

## The Design

<img src=receiver_design.png width=600px />

## The Simulation

- Complex components to design
- Acoustic transport to PZT face
- Constitutive equation solution inside PZT Material (including elastic stress/strain)
- Electrical solution in frequency space (Kirchoff's Laws, complicated by the IC)
- Tools possible - COMSOL (FEM solver), SPICE (circuit solver)

## The Simulation

### COMSOL Multiphysics Simulation

<img src=pzt_comsol_mesh.png width=600px />

## The Simulation

### COMSOL Boundary condition

- Particle Velocity

$$v=1.0\times 10^{-6}\mathrm{\frac{m}{s}} \equiv 20\mathrm{dB}$$

- Conversion to acceleration

$$a = \frac{d}{dt}v=\frac{d}{dt} \left[A \sin \left(2 \pi f t\right)\right]$$
$$a = 2 \pi f A$$

## The Simulation

### COMSOL Voltage Results

In [1]:
# plot voltage result
from pyg.twod import pyg2d
from pym import func as pym
from pyg.colors import pu as puc
import numpy as np

(freq, vel, intvolt, intcurr, maxvolt) = np.loadtxt('pzt_power_harvester.csv', unpack=True, skiprows=5, delimiter=',')

intvoltc = pym.curve(freq, 1.0E6 * intvolt, name='$\int V dx$')
plot = intvoltc.plot(linestyle='-', linecolor=puc.pu_colors['newgold'])
plot.markers_off()
plot.lines_on()
plot.xlabel('Frequency ($f$) [$\mathrm{Hz}]$')
plot.ylabel('Integrated Voltage ($\int V dx$) [$\mathrm{mV \cdot mm}$]')
plot.export('intvolt', sizes=['4'], ratio='silver')
plot.show()

## The Simulation

### COMSOL Current Density Results

In [2]:
# plot current result
intcurrc = pym.curve(freq, 1.0E6 * intcurr, name='$\int I dx$')
plot = intcurrc.plot(linestyle='-', linecolor=puc.pu_colors['teal'])
plot.markers_off()
plot.lines_on()
plot.xlabel(r'Frequency ($f$) [$\mathrm{Hz}]$')
plot.ylabel(r'Integrated Current Density ($\int I dx$) [$\mathrm{\frac{mA}{mm}}$]')
plot.export('intcurr', sizes=['4'], ratio='silver')
plot.show()

In [3]:
# plot power result
intpow = intcurrc * intvoltc
plot = intpow.plot(linestyle='-', linecolor=puc.pu_colors['purple'])
plot.markers_off()
plot.lines_on()
plot.xlim(0, 1e6)
plot.ylim(0, 300)
plot.xlabel(r'Frequency ($f$) [$\mathrm{Hz}]$')
plot.ylabel(r'Integrated Power ($\int P dx$) [$\mathrm{mW}$]')
plot.export('intpow', sizes=['4'], ratio='silver')
plot.show()

## The Simulation

### LTSpice Model

- Illich created
    - a PZT spice component
    - a finite acoustic media component
    - an infinite acoustic media component
- Illich assumed a current $\Leftrightarrow$ particle velocity analogy from Schwarz
- Allows for easy modeling of circuit using pzt component


## The Simulation

### Failed LTSpice Models

<img src=power_harvester_with_lt3020.png width=600px />

- Maximum voltage of $\mathrm{\mu V}$
- Maximum current of $\mathrm{n A}$

## The Simulation

### LTSpice Model

![image of LT spice model](power_harvester_with_lt1585.svg)

## The Simulation

### LTSpice Model Results

In [4]:
# Plot current and voltage from LTSpice
(t, vpwrout, vpztin, ipwrout, ipztin) = np.loadtxt('power_harvester_with_lt1585.txt', unpack=True, skiprows=1)

vpwroutc = pym.curve(1.0E6 * t, vpwrout, name='V$')
plot = vpwroutc.plot(linestyle='-', linecolor=puc.pu_colors['newgold'])
plot.markers_off()
plot.lines_on()
plot.xlabel('Time ($t$) [$\mathrm{\mu s}]$')
plot.ylabel('Voltage ($V$) [$\mathrm{V}$]')
plot.xlim(0, 150)
plot.ylim(0, 4.5)
plot.export('volt', sizes=['4'], ratio='silver')
plot.show()

## The Simulation

### LTSpice Model Results

In [5]:
# Plot current and voltage from LTSpice
(t, vpwrout, vpztin, ipwrout, ipztin) = np.loadtxt('power_harvester_with_lt1585.txt', unpack=True, skiprows=1)

ipwroutc = pym.curve(1.0E6 * t, 1.0E6 * ipwrout, name='$I$')
plot = ipwroutc.plot(linestyle='-', linecolor=puc.pu_colors['teal'])
plot.markers_off()
plot.lines_on()
plot.xlabel('Time ($t$) [$\mathrm{\mu s}]$')
plot.ylabel('Current ($I$) [$\mathrm{\mu A}$]')
plot.xlim(0, 150)
plot.ylim(0, 50)
plot.export('curr', sizes=['4'], ratio='silver')
plot.show()

## The Realization

- Drop in replacement for single cell LiPo battery ($>3.7\mathrm{V}$)
- After start up period of $150\mathrm{\mu s}$, reaches steady state current of $42\mathrm{\mu A}$
- Provides power of $176.4\mathrm{\mu W}$
- Size of $6.6\mathrm{mm^{2}}$

## The Future

- Optimization studies into LC circuit values
- Better PZT material possibilities
- More efficient step-down conversion
- Higher current from higher frequencies ($5\mathrm{MHz}$)
- Design of PCB for size estimates